In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import librosa
import random

# Any results you write to the current directory are saved as output.


[]


In [2]:
def roll(data):
    data_roll = np.roll(data, 5000)
    return data_roll
def stretch(data, rate=2):
    input_length = 16000*3
    data = librosa.effects.time_stretch(data, rate)
    if len(data)>input_length:
        data = data[:input_length]
    else:
        data = np.pad(data, (0, max(0, input_length - len(data))), "constant")

    return data

def wnoise(data):
    wn = np.random.randn(len(data))
    data_wn = data + 0.005*wn
    return data_wn

In [3]:
# put dataset in folder named dataset folder

Cloning into 'dataset'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 1245 (delta 1), reused 146 (delta 1), pack-reused 1098
Receiving objects: 100% (1245/1245), 83.34 MiB | 22.49 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [4]:
import os

def get_all_files(datapath, dataset_type="keyword"):
    data_dir = datapath
    data = []
    all_files = os.listdir(data_dir)
    all_files.remove('.DS_Store')
    labels = set()
    for file in all_files:
        filelabels = file.split("-")[:3]
        data_dict = {
            "filepath": data_dir + file,
            "stress": filelabels[2],
            "environment": filelabels[1],
            "keyword":filelabels[0]
        }
        labels.add(data_dict[dataset_type])
        data.append(data_dict)

    return data,labels

In [5]:
import os
import librosa
import numpy as np
from tqdm import tqdm

class AudioFeatureDataset():

    ''' To create audio dataset
        @param dataset_type = ( keyword | stress | environment   )
    '''

    def __init__(self,datapath, samplingrate=16000, dt="keyword"):
        print(dt)
        datafiles, labels = get_all_files(datapath,dataset_type=dt)
        self.datafiles = datafiles
        self.samplingrate = samplingrate
        self.target_labels = list(labels)
        self.dataset_type = dt

    def process(self, file, max_len=16000):
        ''' extracts raw audio  and returns samps '''
        try:
            samps, sr = librosa.load(file, mono=True, sr=None)
            pad_len = max_len - samps.shape[0]
            if pad_len >= 0:
                samps = np.pad(samps, (0, pad_len), 'constant')
            return np.array(samps[:max_len])
        except:
            print(file)

    def get_dataset(self, include_background=False):
        labels = []
        features = []
        for file_data in tqdm(self.datafiles):
            labels.append(file_data[self.dataset_type])
            samps = self.process(file_data["filepath"], self.samplingrate * 3)
            features.append(samps)
            # with roll
            labels.append(file_data[self.dataset_type])
            features.append(roll(samps)) 
            # with strech
            labels.append(file_data[self.dataset_type])
            features.append(stretch(samps)) 
            # white noise
            labels.append(file_data[self.dataset_type])
            features.append(wnoise(samps)) 
        labels = np.array(labels)
        features = np.array(features)
        return features, labels


In [6]:
a = AudioFeatureDataset(datapath="dataset/data/",dt="keyword")

keyword


In [7]:
features,labels = a.get_dataset()

100%|██████████| 1177/1177 [02:15<00:00,  5.10it/s]


In [8]:
x = np.array(features)
Y = np.array(labels)

In [9]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(Y),Y)

In [10]:
x.shape

(4708, 48000)

In [11]:
x = x.reshape(x.shape[0], 1, 16000 * 3)

In [12]:
df = pd.Series(Y)
target_count = df.value_counts()
print(target_count)

background    1284
unknown       1240
bacho         1124
help          1060
dtype: int64


In [13]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder


labelencoder = LabelEncoder()
Y = labelencoder.fit_transform(Y)

Using TensorFlow backend.


In [14]:
from sklearn.model_selection import train_test_split

def get_train_test(split_ratio=0.7, random_state=42):
    return train_test_split(x, Y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)

In [15]:
X_train, X_test, y_train, y_test = get_train_test()


In [16]:
import tensorflow as tf

In [17]:
tensorboard_callback = tf.keras.callbacks.TensorBoard("logs")


In [18]:
%load_ext tensorboard.notebook
%tensorboard --logdir logs

In [19]:
# y_train = labelencoder.fit_transform(y_train)
mapping = dict(zip(labelencoder.classes_, range(len(labelencoder.classes_))))
mapping

{'bacho': 0, 'background': 1, 'help': 2, 'unknown': 3}

In [20]:
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

In [21]:
!pip install kapre

In [22]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D,SeparableConv2D,BatchNormalization,LSTM,Reshape,TimeDistributed

In [23]:
import keras
import kapre
from keras.models import Sequential
from keras.layers import Dense,AveragePooling2D
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
from kapre.augmentation import AdditiveNoise

# 6 channels (!), maybe 1-sec audio signal, for an example.

sr = 16000
input_shape = (1,sr*3)


def edge_speech_neta():
    tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/esna-100e")
    model = Sequential()
    # A mel-spectrogram layer
    model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                             padding='same', sr=sr, n_mels=128,
                             fmin=0.0, fmax=sr/2, power_melgram=1.0,
                             return_decibel_melgram=True,trainable_fb=False,
                             trainable_kernel=False,
                             name='trainable_stft'))
    # Maybe some additive white noise.
    model.add(AdditiveNoise(power=0.1))
    # If you wanna normalise it per-frequency
    model.add(Normalization2D(str_axis='freq')) # or 'channel', 'time', 'batch', 'data_sample'
    # After this, it's just a usual keras workflow. For example..
    # Add some layers, e.g., model.add(some convolution layers..)
   
    # Compile the model
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu', input_shape=(20, 35, 1)))
    model.add(Conv2D(20, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(15, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(25, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(22, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(22, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(25, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(4, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=1, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    return model

In [24]:
def depth_Seprable_cnn():
    tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/depth-serpable-cnn-100e")
    # Depth Wise CNN (DS-CNN)
    model = Sequential()
    # A mel-spectrogram layer
    model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                             padding='same', sr=sr, n_mels=128,
                             fmin=0.0, fmax=sr/2, power_melgram=1.0,
                             return_decibel_melgram=True,trainable_fb=False,
                             trainable_kernel=False,
                             name='trainable_stft'))
    # Maybe some additive white noise.
    model.add(AdditiveNoise(power=0.1))
    # If you wanna normalise it per-frequency
    model.add(Normalization2D(str_axis='freq')) # or 'channel', 'time', 'batch', 'data_sample'
    # After this, it's just a usual keras workflow. For example..
    # Add some layers, e.g., model.add(some convolution layers..)
    # Compile the model
    model.add(Conv2D(64, kernel_size=(20, 8), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering="th"))
    model.add(Dropout(0.25))
    ## Depth Seprable Pooling Layer - start
    model.add(SeparableConv2D(64, kernel_size=(5, 5), activation='relu',dim_ordering="th"))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(1, 1), activation='relu',dim_ordering="th"))
    model.add(BatchNormalization())
    model.add(SeparableConv2D(64, kernel_size=(5, 5), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(1, 1), activation='relu'))
    model.add(BatchNormalization())
    ## Depth Seprable pooling Layer - end
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(4, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=1, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    return model

In [25]:
def edgespeechnetd():
    tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/edgespeechnet-d-100e")
    model = Sequential()
    # A mel-spectrogram layer
    model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                             padding='same', sr=sr, n_mels=128,
                             fmin=0.0, fmax=sr/2, power_melgram=1.0,
                             return_decibel_melgram=True,trainable_fb=False,
                             trainable_kernel=False,
                             name='trainable_stft'))
    # Maybe some additive white noise.
    model.add(AdditiveNoise(power=0.1))
    # If you wanna normalise it per-frequency
    model.add(Normalization2D(str_axis='freq')) # or 'channel', 'time', 'batch', 'data_sample'
    # After this, it's just a usual keras workflow. For example..
    # Add some layers, e.g., model.add(some convolution layers..)
    # Compile the model
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Conv2D(30, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(33, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(35, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu',dim_ordering="th"))

    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(16))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=50, epochs=50, verbose=1, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    return model

def edgespeechnetb():
    tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/edgespeechnet-b-100e")
    model = Sequential()
    # A mel-spectrogram layer
    model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                             padding='same', sr=sr, n_mels=128,
                             fmin=0.0, fmax=sr/2, power_melgram=1.0,
                             return_decibel_melgram=True,trainable_fb=False,
                             trainable_kernel=False,
                             name='trainable_stft'))
    # Maybe some additive white noise.
    model.add(AdditiveNoise(power=0.1))
    # If you wanna normalise it per-frequency
    model.add(Normalization2D(str_axis='freq')) # or 'channel', 'time', 'batch', 'data_sample'
    # After this, it's just a usual keras workflow. For example..
    # Add some layers, e.g., model.add(some convolution layers..)
    # Compile the model
    model.add(Conv2D(30, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(8, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(30, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(9, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(30, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(11, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(30, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(10, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(30, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(8, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(30, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(11, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(30, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
    model.add(Conv2D(45, kernel_size=(3, 3), activation='relu',dim_ordering="th"))

    model.add(AveragePooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    # model.add(Dense(16))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=50, epochs=50, verbose=1, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    return model

In [26]:
def cnn():
    tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/cnn-100e")
    model = Sequential()
    # A mel-spectrogram layer
    model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                             padding='same', sr=sr, n_mels=128,
                             fmin=0.0, fmax=sr/2, power_melgram=1.0,
                             return_decibel_melgram=True,trainable_fb=False,
                             trainable_kernel=False,
                             name='trainable_stft'))
    # Maybe some additive white noise.
    model.add(AdditiveNoise(power=0.1))
    # If you wanna normalise it per-frequency
    model.add(Normalization2D(str_axis='freq')) # or 'channel', 'time', 'batch', 'data_sample'
    # After this, it's just a usual keras workflow. For example..
    # Add some layers, e.g., model.add(some convolution layers..)
    # Compile the model

    model.add(Conv2D(64, kernel_size=(20, 8), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(10, 4), activation='relu',dim_ordering="th"))
    model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering="th"))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=50, epochs=50, verbose=1, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    model.summary()
    return model

In [27]:
from keras.models import Sequential
from keras.layers import Dense,Permute, Dropout, Flatten, Conv2D, MaxPooling2D,SeparableConv2D,BatchNormalization,AveragePooling2D,GRU,Input

def crnn():
    tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/crnn-50batch")
    model = Sequential()
    model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                             padding='same', sr=sr, n_mels=128,
                             fmin=0.0, fmax=sr/2, power_melgram=1.0,
                             return_decibel_melgram=True,trainable_fb=False,
                             trainable_kernel=False,
                             name='trainable_stft'))
    # Maybe some additive white noise.
    model.add(AdditiveNoise(power=0.1))
    # If you wanna normalise it per-frequency
    model.add(Normalization2D(str_axis='freq'))
    model.add(Conv2D(64, kernel_size=(20, 8), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering="th"))
    model.add(Dropout(0.25))
    ## Depth Seprable Pooling Layer - start
    model.add(SeparableConv2D(64, kernel_size=(5,5), activation='relu',dim_ordering="th"))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(1, 1), activation='relu',dim_ordering="th"))
    model.add(BatchNormalization())
    model.add(SeparableConv2D(64, kernel_size=(5,5), activation='relu',dim_ordering="th"))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(1, 1), activation='relu',dim_ordering="th"))
    model.add(BatchNormalization())
    model.add(SeparableConv2D(64, kernel_size=(5,5), activation='relu',dim_ordering="th"))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(1, 1), activation='relu',dim_ordering="th"))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2),dim_ordering="th"))
    model.add(Reshape((320, -1)))
    model.add(GRU(60, return_sequences=True, name='gru1'))
    model.add(GRU(60, return_sequences=False, name='gru2'))
    model.add(Dropout(0.25))
    model.add(Dense(4, activation='softmax'))
    model.summary()
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.adam(),metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=50, epochs=50, verbose=1, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    return model
    

In [28]:
crnn()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(64, kernel_size=(5, 5), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=(1, 1), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(64, kernel_size=(5, 5), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=(1, 1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
trainable_stft (Melspectrogr (None, 128, 94, 1)        296064    
_________________________________________________________________
additive_noise_1 (AdditiveNo (None, 128, 94, 1)        0         
_________________________________________________________________
normalization2d_1 (Normaliza (None, 128, 94, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 87, 64)       10304     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 109, 43, 32)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 109, 43, 32)       0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 64, 39, 28)        9765      
__________

In [29]:
edge_speech_neta()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(39, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(15, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(39, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, kernel_size=(3, 3), activation="relu", d

Train on 3295 samples, validate on 1413 samples
Epoch 1/100
3295/3295 [==============================] - 6s 2ms/step - loss: 1.3050 - acc: 0.3363 - val_loss: 1.1813 - val_acc: 0.3970
Epoch 2/100
3295/3295 [==============================] - 4s 1ms/step - loss: 1.0744 - acc: 0.4971 - val_loss: 1.0256 - val_acc: 0.5159
Epoch 3/100
3295/3295 [==============================] - 4s 1ms/step - loss: 1.0486 - acc: 0.4950 - val_loss: 1.0677 - val_acc: 0.5046
Epoch 4/100
3295/3295 [==============================] - 4s 1ms/step - loss: 1.0687 - acc: 0.5014 - val_loss: 1.0202 - val_acc: 0.5258
Epoch 5/100
3295/3295 [==============================] - 4s 1ms/step - loss: 0.9966 - acc: 0.5426 - val_loss: 1.0084 - val_acc: 0.5159
Epoch 6/100
3295/3295 [==============================] - 4s 1ms/step - loss: 0.9531 - acc: 0.5848 - val_loss: 0.8983 - val_acc: 0.6178
Epoch 7/100
3295/3295 [==============================] - 4s 1ms/step - loss: 0.8613 - acc: 0.6282 - val_loss: 0.8381 - val_acc: 0.6716
Epoch 8

In [30]:
depth_Seprable_cnn()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(64, kernel_size=(5, 5), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=(1, 1), activation="relu", data_format="channels_first")`


Train on 3295 samples, validate on 1413 samples
Epoch 1/100
3295/3295 [==============================] - 7s 2ms/step - loss: 2.4717 - acc: 0.4710 - val_loss: 2.6356 - val_acc: 0.4798
Epoch 2/100
3295/3295 [==============================] - 5s 2ms/step - loss: 1.5875 - acc: 0.5821 - val_loss: 1.1882 - val_acc: 0.5938
Epoch 3/100
3295/3295 [==============================] - 5s 1ms/step - loss: 1.0617 - acc: 0.6340 - val_loss: 1.0758 - val_acc: 0.6695
Epoch 4/100
3295/3295 [==============================] - 5s 2ms/step - loss: 0.8090 - acc: 0.7129 - val_loss: 0.9479 - val_acc: 0.7431
Epoch 5/100
3295/3295 [==============================] - 5s 1ms/step - loss: 0.7288 - acc: 0.7527 - val_loss: 0.9240 - val_acc: 0.7516
Epoch 6/100
3295/3295 [==============================] - 5s 2ms/step - loss: 0.5959 - acc: 0.7933 - val_loss: 0.6571 - val_acc: 0.8266
Epoch 7/100
3295/3295 [==============================] - 5s 1ms/step - loss: 0.4767 - acc: 0.8458 - val_loss: 0.6050 - val_acc: 0.8266
Epoch 8

In [31]:
edgespeechnetb()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(30, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(9, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:57: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(30, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(11, kernel_size=(3, 3), activation="relu", dat

Train on 3295 samples, validate on 1413 samples
Epoch 1/50
3295/3295 [==============================] - 5s 1ms/step - loss: 1.3826 - acc: 0.2816 - val_loss: 1.3660 - val_acc: 0.2626
Epoch 2/50
3295/3295 [==============================] - 2s 742us/step - loss: 1.3439 - acc: 0.2713 - val_loss: 1.3844 - val_acc: 0.2689
Epoch 3/50
3295/3295 [==============================] - 2s 731us/step - loss: 1.3845 - acc: 0.2695 - val_loss: 1.3845 - val_acc: 0.2689
Epoch 4/50
3295/3295 [==============================] - 2s 734us/step - loss: 1.3837 - acc: 0.2744 - val_loss: 1.3847 - val_acc: 0.2689
Epoch 5/50
3295/3295 [==============================] - 2s 736us/step - loss: 1.3838 - acc: 0.2616 - val_loss: 1.3843 - val_acc: 0.2689
Epoch 6/50
3295/3295 [==============================] - 2s 739us/step - loss: 1.3834 - acc: 0.2744 - val_loss: 1.3852 - val_acc: 0.2689
Epoch 7/50
3295/3295 [==============================] - 2s 734us/step - loss: 1.3836 - acc: 0.2601 - val_loss: 1.3844 - val_acc: 0.2689
Ep

In [32]:
edgespeechnetd()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(30, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(45, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(33, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(45, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(35, kernel_size=(3, 3), activation="relu", d

Train on 3295 samples, validate on 1413 samples
Epoch 1/50
3295/3295 [==============================] - 5s 1ms/step - loss: 1.2184 - acc: 0.4076 - val_loss: 1.0761 - val_acc: 0.4317
Epoch 2/50
3295/3295 [==============================] - 3s 801us/step - loss: 0.9549 - acc: 0.5624 - val_loss: 0.9161 - val_acc: 0.5924
Epoch 3/50
3295/3295 [==============================] - 3s 797us/step - loss: 0.8727 - acc: 0.6231 - val_loss: 0.8135 - val_acc: 0.6617
Epoch 4/50
3295/3295 [==============================] - 3s 796us/step - loss: 0.7863 - acc: 0.6744 - val_loss: 0.8460 - val_acc: 0.6992
Epoch 5/50
3295/3295 [==============================] - 3s 796us/step - loss: 0.6516 - acc: 0.7539 - val_loss: 0.6051 - val_acc: 0.7849
Epoch 6/50
3295/3295 [==============================] - 3s 796us/step - loss: 0.5468 - acc: 0.8036 - val_loss: 0.5859 - val_acc: 0.7757
Epoch 7/50
3295/3295 [==============================] - 3s 797us/step - loss: 0.4808 - acc: 0.8252 - val_loss: 0.6021 - val_acc: 0.8125
Ep

In [33]:
cnn()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=(10, 4), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`


Train on 3295 samples, validate on 1413 samples
Epoch 1/50
3295/3295 [==============================] - 5s 1ms/step - loss: 1.2302 - acc: 0.4291 - val_loss: 1.0426 - val_acc: 0.5032
Epoch 2/50
3295/3295 [==============================] - 3s 812us/step - loss: 0.9511 - acc: 0.5642 - val_loss: 0.9083 - val_acc: 0.6001
Epoch 3/50
3295/3295 [==============================] - 3s 818us/step - loss: 0.9162 - acc: 0.5839 - val_loss: 0.9229 - val_acc: 0.6051
Epoch 4/50
3295/3295 [==============================] - 3s 810us/step - loss: 0.9210 - acc: 0.5833 - val_loss: 0.9572 - val_acc: 0.5718
Epoch 5/50
3295/3295 [==============================] - 3s 812us/step - loss: 0.8565 - acc: 0.6303 - val_loss: 0.8060 - val_acc: 0.6752
Epoch 6/50
3295/3295 [==============================] - 3s 811us/step - loss: 0.7428 - acc: 0.6874 - val_loss: 0.6987 - val_acc: 0.7431
Epoch 7/50
3295/3295 [==============================] - 3s 814us/step - loss: 0.6401 - acc: 0.7469 - val_loss: 0.6390 - val_acc: 0.7466
Ep

In [34]:
!rm -rf dataset

In [35]:
!ls

__notebook__.ipynb  __output__.json  logs


In [36]:
!rm -rf logs

In [37]:
!ls

__notebook__.ipynb  __output__.json


In [38]:
print(modl)

NameError: name 'modl' is not defined